<a 
href="https://colab.research.google.com/github/wingated/cs474_labs_f2019/blob/master/DL_Lab7.ipynb"
  target="_parent">
  <img
    src="https://colab.research.google.com/assets/colab-badge.svg"
    alt="Open In Colab"/>
</a>

# Music Transformer
### Description:
This notebook contains the architecture for a decoder-only transformer. It can be used to generate a long sequence of any type of token, but in this case, it was used for music notes. Its results can be heard here: https://soundcloud.com/joshbtay/sets/transformer-results. In a previous iteration, I attempted to use an LSTM to generate music. Its results can be heard here: https://soundcloud.com/joshbtay/sets/lstm-generative-results

#### Based on "Attention is All You Need" (https://arxiv.org/abs/1706.03762) and "The Annotated Transformer" (https://nlp.seas.harvard.edu/2018/04/03/attention.html), altered to use decoder-only structure



## Setup


In [4]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math, copy, time
from torch.autograd import Variable
import matplotlib.pyplot as plt
import seaborn
seaborn.set_context(context="talk")
%matplotlib inline

## Model Helpers


In [3]:
class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))
    
class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)
    
class Generator(nn.Module):
    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)

    def forward(self, x):
        return F.log_softmax(self.proj(x), dim=-1)

class LayerNorm(nn.Module):
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

class SublayerConnection(nn.Module):
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        return x + self.dropout(sublayer(self.norm(x)))
    
def clone(module, N):
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

## Decoder

The decoder is also composed of a stack of $N=6$ identical layers.  


In [5]:
class Decoder(nn.Module):
    def __init__(self, layer, N):
        super(Decoder, self).__init__()
        self.layers = clone(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, tgt_mask):
        for layer in self.layers:
            x = layer(x, tgt_mask)
        return self.norm(x)
    
class DecoderLayer(nn.Module):
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(DecoderLayer, self).__init__()
        self.size = size
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clone(SublayerConnection(size, dropout), 3)
 
    def forward(self, x, tgt_mask):
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
        return self.sublayer[2](x, self.feed_forward)
    
def subsequent_mask(size):
    attn_shape = (1, size, size)
    subsequent_mask = np.triu(np.ones(attn_shape), k=1).astype('uint8')
    return torch.from_numpy(subsequent_mask) == 0

In [6]:
def attention(query, key, value, mask):
    scale = np.sqrt(key.shape[2])
    scores = torch.bmm(query, key.permute(0, 2, 1)) / scale
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    output = torch.bmm(F.softmax(scores, dim = 2), value)
    return output

In [7]:
class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        super(MultiHeadedAttention, self).__init__()
        self.dropout = dropout
        head = nn.Linear(d_model, d_model//h).cuda()
        self.attention_heads = nn.ModuleList([clone(head, 3) for _ in range(h)])
        self.w_output: nn.Linear = nn.Linear(d_model, d_model)

    def forward(self, query, key, value, mask):
        output = []
        for head in self.attention_heads:
            q = head[0](query)
            k = head[1](key)
            v = head[2](value)
            output.append(attention(q, k, v, mask))
        outputs_concat = torch.cat(output, 2)
        multi_head = self.w_output(outputs_concat)
        return multi_head

In [8]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = 1 / (10000 ** (torch.arange(0., d_model, 2) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + Variable(self.pe[:, :x.size(1)], 
                         requires_grad=False)
        return self.dropout(x)

## Full Model

In [13]:
class TransformerModel(nn.Module):
    def __init__(self, tgt_vocab, N=6, d_model=256, d_ff=1024, h=8, dropout=0.1):
        super(TransformerModel, self).__init__()
        
        attn = MultiHeadedAttention(h, d_model)
        ff = PositionwiseFeedForward(d_model, d_ff, dropout)
        position = PositionalEncoding(d_model, dropout)
        c = copy.deepcopy
        
        self.decoder = Decoder(DecoderLayer(d_model, c(attn), c(ff), dropout), N)
        self.tgt_embed = nn.Sequential(Embeddings(d_model, tgt_vocab), c(position))
        self.generator = Generator(d_model, tgt_vocab)
        for p in self.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)
        
    def forward(self, tgt, tgt_mask):
        return self.decode(tgt, tgt_mask)
    
    def decode(self, tgt, tgt_mask):
        return self.decoder(self.tgt_embed(tgt), tgt_mask)

# Training


## Batches and Masking

In [14]:
class Batch:
    def __init__(self, trg=None, pad=0):
        if trg is not None:
            self.trg = trg[:, :-1]
            self.trg_y = trg[:, 1:]
            self.trg_mask = self.make_std_mask(self.trg, pad)
            self.ntokens = (self.trg_y != pad).data.sum()
    
    @staticmethod
    def make_std_mask(tgt, pad):
        tgt_mask = (tgt != pad).unsqueeze(-2)
        tgt_mask = tgt_mask & Variable(
            subsequent_mask(tgt.size(-1)).type_as(tgt_mask.data))
        return tgt_mask
    
    
global max_tgt_in_batch
def batch_size_fn(new, count, sofar):
    global max_tgt_in_batch
    if count == 1:
        max_tgt_in_batch = 0
    max_tgt_in_batch = max(max_tgt_in_batch,  len(new.trg) + 2)
    tgt_elements = count * max_tgt_in_batch
    return tgt_elements

In [15]:
class LabelSmoothing(nn.Module):
    def __init__(self, size, padding_idx, smoothing=0.0):
        super(LabelSmoothing, self).__init__()
        self.criterion = nn.KLDivLoss(reduction='sum')
        self.padding_idx = padding_idx
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.size = size
        self.true_dist = None
        
    def forward(self, x, target):
        assert x.size(1) == self.size
        true_dist = x.data.clone()
        true_dist.fill_(self.smoothing / (self.size - 2))
        true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        true_dist[:, self.padding_idx] = 0
        mask = torch.nonzero(target.data == self.padding_idx)
        if mask.dim() > 0:
            true_dist.index_fill_(0, mask.squeeze(), 0.0)
        self.true_dist = true_dist
        return self.criterion(x, Variable(true_dist, requires_grad=False))

## Data Loading


In [16]:
from torchtext.legacy import data, datasets
import torchtext

def tokenize(text):
    return text.split()

BOS_WORD = '<s>'
EOS_WORD = '</s>'
BLANK_WORD = "<blank>"
TGT = data.Field(tokenize=tokenize, init_token = BOS_WORD, eos_token = EOS_WORD, pad_token=BLANK_WORD)

print("Loading Dataset")
music_text = open('./data.txt', 'r')
music_lines = list(music_text)

fields = [("trg", TGT)]
examples = [torchtext.legacy.data.Example.fromlist([(music_lines[i])], fields) for i in range(len(music_lines))]

MAX_LEN = 1500
train, val = torchtext.legacy.data.Dataset(examples, fields=fields, filter_pred=lambda x: 
        len(vars(x)['trg']) <= MAX_LEN).split()

MIN_FREQ = 1
TGT.build_vocab(train.trg, min_freq=MIN_FREQ)

Loading Dataset


## Training Code

In [17]:
class LossFunction:
    def __init__(self, generator, criterion, opt=None):
        self.generator = generator
        self.criterion = criterion
        self.opt = opt
        
    def __call__(self, x, y, norm):
        x = self.generator(x)
        loss = self.criterion(x.contiguous().view(-1, x.size(-1)), 
                              y.contiguous().view(-1)) / norm
        loss.backward()
        if self.opt is not None:
            self.opt.step()
            self.opt.zero_grad()
        return loss.data * norm

class DataIterator(data.Iterator):
    def create_batches(self):
        if self.train:
            def pool(d, random_shuffler):
                for p in data.batch(d, self.batch_size * 100):
                    p_batch = data.batch(
                        sorted(p, key=self.sort_key),
                        self.batch_size, self.batch_size_fn)
                    for b in random_shuffler(list(p_batch)):
                        yield b
            self.batches = pool(self.data(), self.random_shuffler)
            
        else:
            self.batches = []
            for b in data.batch(self.data(), self.batch_size,
                                          self.batch_size_fn):
                self.batches.append(sorted(b, key=self.sort_key))

def rebatch(pad_idx, batch):
    trg = batch.trg.transpose(0, 1).cuda()
    return Batch(trg, pad_idx)

    
def run_epoch(data_iter, model, loss_compute, startt):
    start = time.time()
    total_tokens = 0
    total_loss = 0
    tokens = 0
    for i, batch in enumerate(data_iter):
        if time.time() - startt > timeout:
          break
        out = model.forward(batch.trg, batch.trg_mask)
        loss = loss_compute(out, batch.trg_y, batch.ntokens)
        total_loss += loss
        total_tokens += batch.ntokens
        tokens += batch.ntokens
        if i % 50 == 1:
            elapsed = time.time() - start
            print("Epoch Step: %d Loss: %f Tokens per Sec: %f Time elapsed: %f" %
                    (i, loss / batch.ntokens, tokens / elapsed, time.time() - startt))
            start = time.time()
            tokens = 0
    return total_loss / total_tokens
    

##Train

In [8]:
import gc
gc.collect()
timeout = 14400
pad_idx = TGT.vocab.stoi["<blank>"]
model = TransformerModel(len(TGT.vocab), N=8).cuda()
n_epochs = 10000
device = torch.device('cuda')

In [56]:
import gc
gc.collect()
timeout = 14400
pad_idx = TGT.vocab.stoi["<blank>"]
model = TransformerModel(len(TGT.vocab), N=8).cuda()
n_epochs = 10000
device = torch.device('cuda')

def scope():
    criterion = LabelSmoothing(size=len(TGT.vocab), padding_idx=pad_idx, smoothing=0.1)
    criterion.cuda()
    BATCH_SIZE = 1000
    train_iter = DataIterator(train, batch_size=BATCH_SIZE, device=device,
                            repeat=False, sort_key=lambda x: (len(x.trg)),
batch_size_fn=batch_size_fn, train=True)
    valid_iter = DataIterator(val, batch_size=BATCH_SIZE, device=device,
                            repeat=False, sort_key=lambda x: (len(x.trg)),
                            batch_size_fn=batch_size_fn, train=False)

    model_opt = torch.optim.Adam(model.parameters(), lr=5e-4)
    start = time.time()
    for epoch in range(n_epochs):
        if time.time() - start > timeout:
          break
        model.train()
        run_epoch((rebatch(pad_idx, b) for b in train_iter), 
                  model, 
                  LossFunction(model.generator, criterion, model_opt), start)
        model.eval()
scope()

Epoch Step: 1 Loss: 7.663717 Tokens per Sec: 8264.153320 Time elapsed: 0.175374
Epoch Step: 51 Loss: 4.005707 Tokens per Sec: 8908.946289 Time elapsed: 5.303614
Epoch Step: 101 Loss: 5.866451 Tokens per Sec: 8934.105469 Time elapsed: 10.212579
Epoch Step: 151 Loss: 5.384609 Tokens per Sec: 8834.098633 Time elapsed: 15.244917
Epoch Step: 201 Loss: 4.066658 Tokens per Sec: 8793.038086 Time elapsed: 20.574724
Epoch Step: 251 Loss: 4.971523 Tokens per Sec: 9119.349609 Time elapsed: 25.578828
Epoch Step: 301 Loss: 5.090078 Tokens per Sec: 8802.194336 Time elapsed: 31.018290
Epoch Step: 351 Loss: 4.540069 Tokens per Sec: 9068.074219 Time elapsed: 35.953102
Epoch Step: 401 Loss: 4.870378 Tokens per Sec: 8978.536133 Time elapsed: 40.723723
Epoch Step: 451 Loss: 4.168019 Tokens per Sec: 9083.012695 Time elapsed: 45.776695
Epoch Step: 501 Loss: 4.822306 Tokens per Sec: 8910.863281 Time elapsed: 51.226589
Epoch Step: 551 Loss: 4.142798 Tokens per Sec: 8894.094727 Time elapsed: 56.201025
Epoch Ste

Epoch Step: 451 Loss: 3.148880 Tokens per Sec: 8791.461914 Time elapsed: 502.575223
Epoch Step: 501 Loss: 4.004881 Tokens per Sec: 9094.595703 Time elapsed: 507.547428
Epoch Step: 551 Loss: 3.900312 Tokens per Sec: 9003.416016 Time elapsed: 512.410061
Epoch Step: 601 Loss: 4.788691 Tokens per Sec: 8708.364258 Time elapsed: 517.767916
Epoch Step: 651 Loss: 2.969686 Tokens per Sec: 8641.366211 Time elapsed: 523.264401
Epoch Step: 701 Loss: 4.338026 Tokens per Sec: 8823.607422 Time elapsed: 528.442793
Epoch Step: 751 Loss: 3.989018 Tokens per Sec: 8800.588867 Time elapsed: 533.388739
Epoch Step: 801 Loss: 3.649264 Tokens per Sec: 8779.349609 Time elapsed: 538.499271
Epoch Step: 851 Loss: 3.128394 Tokens per Sec: 8752.495117 Time elapsed: 543.788528
Epoch Step: 901 Loss: 3.530771 Tokens per Sec: 8844.401367 Time elapsed: 548.650047
Epoch Step: 951 Loss: 3.773097 Tokens per Sec: 8717.601562 Time elapsed: 553.855059
Epoch Step: 1001 Loss: 3.959375 Tokens per Sec: 8646.996094 Time elapsed: 55

Epoch Step: 851 Loss: 3.614149 Tokens per Sec: 8708.919922 Time elapsed: 1002.483114
Epoch Step: 901 Loss: 3.168075 Tokens per Sec: 8827.637695 Time elapsed: 1007.863751
Epoch Step: 951 Loss: 3.874890 Tokens per Sec: 8475.737305 Time elapsed: 1013.013046
Epoch Step: 1001 Loss: 2.890341 Tokens per Sec: 8563.395508 Time elapsed: 1018.429393
Epoch Step: 1051 Loss: 3.634896 Tokens per Sec: 8615.934570 Time elapsed: 1023.863646
Epoch Step: 1101 Loss: 3.259707 Tokens per Sec: 8429.695312 Time elapsed: 1029.339536
Epoch Step: 1151 Loss: 3.370531 Tokens per Sec: 8818.476562 Time elapsed: 1034.313219
Epoch Step: 1201 Loss: 2.547144 Tokens per Sec: 8802.104492 Time elapsed: 1039.419523
Epoch Step: 1251 Loss: 3.362764 Tokens per Sec: 8956.738281 Time elapsed: 1044.236713
Epoch Step: 1301 Loss: 4.091253 Tokens per Sec: 8872.170898 Time elapsed: 1049.402098
Epoch Step: 1351 Loss: 3.548906 Tokens per Sec: 8861.846680 Time elapsed: 1054.623030
Epoch Step: 1401 Loss: 2.073373 Tokens per Sec: 8746.0888

Epoch Step: 1201 Loss: 3.022398 Tokens per Sec: 9271.718750 Time elapsed: 1496.564469
Epoch Step: 1251 Loss: 3.022870 Tokens per Sec: 8977.366211 Time elapsed: 1501.643941
Epoch Step: 1301 Loss: 3.699251 Tokens per Sec: 8796.477539 Time elapsed: 1506.564654
Epoch Step: 1351 Loss: 3.711789 Tokens per Sec: 8968.469727 Time elapsed: 1511.506099
Epoch Step: 1401 Loss: 2.285858 Tokens per Sec: 8970.640625 Time elapsed: 1516.570203
Epoch Step: 1451 Loss: 3.571016 Tokens per Sec: 8758.775391 Time elapsed: 1521.661225
Epoch Step: 1501 Loss: 2.876711 Tokens per Sec: 9048.950195 Time elapsed: 1526.497942
Epoch Step: 1551 Loss: 1.476979 Tokens per Sec: 9071.973633 Time elapsed: 1531.481115
Epoch Step: 1601 Loss: 3.073004 Tokens per Sec: 9006.794922 Time elapsed: 1536.263195
Epoch Step: 1651 Loss: 2.366772 Tokens per Sec: 8950.005859 Time elapsed: 1541.488357
Epoch Step: 1701 Loss: 2.671904 Tokens per Sec: 8606.743164 Time elapsed: 1546.772842
Epoch Step: 1751 Loss: 4.550711 Tokens per Sec: 9153.0

Epoch Step: 1551 Loss: 3.239641 Tokens per Sec: 9010.713867 Time elapsed: 1979.749491
Epoch Step: 1601 Loss: 3.180445 Tokens per Sec: 8889.759766 Time elapsed: 1984.802636
Epoch Step: 1651 Loss: 2.202291 Tokens per Sec: 8914.108398 Time elapsed: 1989.800590
Epoch Step: 1701 Loss: 2.861981 Tokens per Sec: 9045.174805 Time elapsed: 1994.946996
Epoch Step: 1751 Loss: 2.693207 Tokens per Sec: 8890.032227 Time elapsed: 2000.135421
Epoch Step: 1801 Loss: 2.617398 Tokens per Sec: 9192.189453 Time elapsed: 2004.859246
Epoch Step: 1851 Loss: 3.313418 Tokens per Sec: 8891.481445 Time elapsed: 2009.791638
Epoch Step: 1901 Loss: 4.251902 Tokens per Sec: 9062.144531 Time elapsed: 2014.543729
Epoch Step: 1951 Loss: 3.129068 Tokens per Sec: 8947.836914 Time elapsed: 2019.717734
Epoch Step: 2001 Loss: 3.002727 Tokens per Sec: 9161.800781 Time elapsed: 2024.662750
Epoch Step: 2051 Loss: 4.005679 Tokens per Sec: 8237.166016 Time elapsed: 2029.972373
Epoch Step: 2101 Loss: 3.936482 Tokens per Sec: 8233.5

...

Epoch Step: 4251 Loss: 0.662812 Tokens per Sec: 8201.515625 Time elapsed: 12137.969831
Epoch Step: 4301 Loss: 0.973542 Tokens per Sec: 8175.985840 Time elapsed: 12143.654936
Epoch Step: 4351 Loss: 0.793273 Tokens per Sec: 8304.037109 Time elapsed: 12149.383105
Epoch Step: 4401 Loss: 0.849041 Tokens per Sec: 8100.708984 Time elapsed: 12154.382306
Epoch Step: 1 Loss: 1.910338 Tokens per Sec: 8108.907715 Time elapsed: 12155.619479
Epoch Step: 51 Loss: 1.790358 Tokens per Sec: 8462.892578 Time elapsed: 12160.863814
Epoch Step: 101 Loss: 1.296848 Tokens per Sec: 8592.963867 Time elapsed: 12166.196047
Epoch Step: 151 Loss: 0.601888 Tokens per Sec: 8424.317383 Time elapsed: 12171.517573
Epoch Step: 201 Loss: 0.557513 Tokens per Sec: 8678.663086 Time elapsed: 12176.722640
Epoch Step: 251 Loss: 1.480501 Tokens per Sec: 8640.200195 Time elapsed: 12182.279007
Epoch Step: 301 Loss: 0.793334 Tokens per Sec: 8766.037109 Time elapsed: 12187.587626
Epoch Step: 351 Loss: 1.469830 Tokens per Sec: 8509.9

Epoch Step: 101 Loss: 1.447974 Tokens per Sec: 8673.423828 Time elapsed: 12627.721125
Epoch Step: 151 Loss: 0.631458 Tokens per Sec: 8563.148438 Time elapsed: 12633.214457
Epoch Step: 201 Loss: 1.312929 Tokens per Sec: 8592.009766 Time elapsed: 12638.497165
Epoch Step: 251 Loss: 0.206106 Tokens per Sec: 8884.775391 Time elapsed: 12643.712055
Epoch Step: 301 Loss: 1.900539 Tokens per Sec: 8927.886719 Time elapsed: 12648.663428
Epoch Step: 351 Loss: 0.699104 Tokens per Sec: 8888.028320 Time elapsed: 12653.361322
Epoch Step: 401 Loss: 1.308396 Tokens per Sec: 8979.989258 Time elapsed: 12658.475946
Epoch Step: 451 Loss: 0.963333 Tokens per Sec: 8829.238281 Time elapsed: 12663.701223
Epoch Step: 501 Loss: 0.484200 Tokens per Sec: 8906.823242 Time elapsed: 12668.548769
Epoch Step: 551 Loss: 2.470863 Tokens per Sec: 8901.640625 Time elapsed: 12673.610231
Epoch Step: 601 Loss: 0.551816 Tokens per Sec: 8827.500000 Time elapsed: 12678.707297
Epoch Step: 651 Loss: 0.969275 Tokens per Sec: 8694.36

Epoch Step: 401 Loss: 1.354475 Tokens per Sec: 8856.127930 Time elapsed: 13105.688846
Epoch Step: 451 Loss: 0.994959 Tokens per Sec: 9096.741211 Time elapsed: 13110.704961
Epoch Step: 501 Loss: 0.299316 Tokens per Sec: 9050.209961 Time elapsed: 13115.714383
Epoch Step: 551 Loss: 0.462187 Tokens per Sec: 8793.893555 Time elapsed: 13120.987715
Epoch Step: 601 Loss: 1.241275 Tokens per Sec: 9191.236328 Time elapsed: 13125.830758
Epoch Step: 651 Loss: 3.290330 Tokens per Sec: 9198.523438 Time elapsed: 13130.716789
Epoch Step: 701 Loss: 1.486030 Tokens per Sec: 9184.686523 Time elapsed: 13135.617262
Epoch Step: 751 Loss: 1.127875 Tokens per Sec: 9197.497070 Time elapsed: 13140.444052
Epoch Step: 801 Loss: 0.676164 Tokens per Sec: 9046.861328 Time elapsed: 13145.797018
Epoch Step: 851 Loss: 1.646084 Tokens per Sec: 9128.144531 Time elapsed: 13150.708006
Epoch Step: 901 Loss: 1.382466 Tokens per Sec: 9033.731445 Time elapsed: 13155.405764
Epoch Step: 951 Loss: 2.126590 Tokens per Sec: 8921.69

Epoch Step: 701 Loss: 0.637887 Tokens per Sec: 9142.495117 Time elapsed: 13580.470098
Epoch Step: 751 Loss: 1.229605 Tokens per Sec: 8948.051758 Time elapsed: 13585.816348
Epoch Step: 801 Loss: 0.441736 Tokens per Sec: 8907.240234 Time elapsed: 13591.197055
Epoch Step: 851 Loss: 0.585512 Tokens per Sec: 8811.000977 Time elapsed: 13596.474795
Epoch Step: 901 Loss: 2.229995 Tokens per Sec: 9242.750977 Time elapsed: 13601.313101
Epoch Step: 951 Loss: 1.894708 Tokens per Sec: 9240.012695 Time elapsed: 13605.850887
Epoch Step: 1001 Loss: 0.946942 Tokens per Sec: 9071.542969 Time elapsed: 13611.177230
Epoch Step: 1051 Loss: 1.787283 Tokens per Sec: 9165.373047 Time elapsed: 13616.044166
Epoch Step: 1101 Loss: 1.375503 Tokens per Sec: 9123.331055 Time elapsed: 13621.008613
Epoch Step: 1151 Loss: 0.891441 Tokens per Sec: 8988.241211 Time elapsed: 13626.020079
Epoch Step: 1201 Loss: 2.472004 Tokens per Sec: 9054.466797 Time elapsed: 13631.216327
Epoch Step: 1251 Loss: 1.985297 Tokens per Sec: 9

Epoch Step: 1001 Loss: 3.547288 Tokens per Sec: 9216.456055 Time elapsed: 14056.061270
Epoch Step: 1051 Loss: 1.186712 Tokens per Sec: 9226.693359 Time elapsed: 14060.696980
Epoch Step: 1101 Loss: 0.830216 Tokens per Sec: 8998.165039 Time elapsed: 14065.561778
Epoch Step: 1151 Loss: 1.501654 Tokens per Sec: 9024.202148 Time elapsed: 14070.415081
Epoch Step: 1201 Loss: 0.944452 Tokens per Sec: 9043.666992 Time elapsed: 14075.693389
Epoch Step: 1251 Loss: 0.930512 Tokens per Sec: 8921.464844 Time elapsed: 14080.989518
Epoch Step: 1301 Loss: 1.185051 Tokens per Sec: 8830.823242 Time elapsed: 14086.193020
Epoch Step: 1351 Loss: 0.908426 Tokens per Sec: 9013.037109 Time elapsed: 14090.891686
Epoch Step: 1401 Loss: 1.159527 Tokens per Sec: 9079.817383 Time elapsed: 14095.854922
Epoch Step: 1451 Loss: 1.176605 Tokens per Sec: 9013.998047 Time elapsed: 14101.172780
Epoch Step: 1501 Loss: 1.781942 Tokens per Sec: 8994.155273 Time elapsed: 14106.377518
Epoch Step: 1551 Loss: 0.525579 Tokens per 

## Translate

In [ ]:
import random
def greedy_decode(model, src, max_len, start_symbol, start_seq):
    k=20
    ys = torch.ones(1, 1).fill_(start_symbol).type_as(src.data)
    for sym in start_seq:
        ys = torch.cat([ys, torch.ones(1, 1).type_as(src.data).fill_(sym)], dim=1)
    for i in range(max_len-1):
        out = model.decode(Variable(ys), Variable(subsequent_mask(ys.size(1)).type_as(src.data)))
        prob = model.generator(out[:, -1])
        dist, indices = prob[0].sort()
        norm = []
        dist = dist[-k:]
        indices = indices[-k:]
        mi = min(dist)
        ma = max(dist)
        s = 0
        for j in dist:
          norm.append((j - mi)/(ma-mi))
          s+= norm[-1]
        p = random.random() ** 2 * s
        #p = random.uniform(0, s)
        s = 0
        for j,q in enumerate(norm):
          s += q
          if s >= p:
            next_word = indices[j]
            break
        #_, next_word = torch.max(prob, dim = 1)
        #next_word = next_word.data[0]
        ys = torch.cat([ys, torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=1)
    return ys

BATCH_SIZE = 1000
n_train_iters = len(train) / BATCH_SIZE
valid_iter = DataIterator(val, batch_size=BATCH_SIZE, device=device,
                        repeat=False, sort_key=lambda x: (len(x.trg)),
                        batch_size_fn=batch_size_fn, train=False)
    
for i, batch in enumerate(valid_iter):
    start_seq = []
    for j in range(1, 5):
       start_seq.append(batch.trg.data[j, 0])
    src = batch.trg.transpose(0, 1)[:1].cuda()
    out = greedy_decode(model, src, max_len=200, start_symbol=TGT.vocab.stoi["<s>"], start_seq=start_seq)
    print("Translation:", end="\t")
    f = open(f"Outmidis/{i:04}.txt", "w")
    for j in range(1, out.size(1)):
        sym = TGT.vocab.itos[out[0, j]]
        if sym == "</s>": break
        print(sym, end =" ")
        f.write(sym+ " ")
    f.close()
    print()
    print("Target:\t", end="\t")
    for j in range(1, batch.trg.size(0)):
        sym = TGT.vocab.itos[batch.trg.data[j, 0]]
        if sym == "</s>": break
        print(sym, end =" ")
    print()
    print()
    
    if i > 1000 and i<1100:
        break

Translation:	0:36 0:55 0:60 0:64 6:48 12:55 12:55 6:60 0:67 6:64 0:72 6:74 6:48 12:55 12:64 12:67 12:36 0:58 0:63 0:67 6:48 12:55 12:48 12:36 0:58 0:60 0:64 0:67 6:48 12:52 0:55 12:53 0:57 12:65 12:48 0:69 12:69 6:36 0:67 6:53 0:63 12:60 0:65 6:36 0:64 12:55 12:55 12:55 12:55 0:60 12:48 0:55 12:64 12:53 0:70 12:72 6:72 6:53 0:56 12:51 0:68 12:53 0:70 6:68 6:58 0:67 12:67 12:36 12:55 12:64 6:67 6:48 0:72 12:60 12:58 6:63 0:67 6:48 12:36 0:64 12:64 6:48 0:65 0:74 12:63 12:43 0:69 12:62 12:41 0:60 0:69 12:48 12:36 0:67 12:64 12:67 12:41 0:70 0:72 12:70 12:45 0:74 12:48 0:69 12:69 12:38 0:70 12:60 12:40 0:71 12:40 0:70 12:58 12:52 0:61 12:53 12:56 12:48 0:69 12:57 12:60 12:45 12:48 0:67 12:64 12:46 0:61 0:67 12:46 12:39 12:46 12:43 0:67 12:46 0:67 12:39 12:63 12:41 0:60 12:69 12:40 0:69 12:41 12:60 12:50 12:43 0:62 12:58 12:62 12:41 0:63 0:68 12:43 12:41 0:69 12:64 12:43 0:70 12:40 0:73 12:53 12:44 0:65 12:49 0:70 12:46 0:61 0:65 12:48 0:63 12:46 0:63 12:48 0:63 12:49 0:67 12:51 0:65 6:67 

Translation:	0:55 12:48 0:64 24:55 12:67 12:62 12:69 12:72 12:67 12:60 0:69 12:53 0:69 12:48 0:71 12:67 12:71 12:64 12:69 12:55 0:62 12:43 0:65 0:67 12:69 12:43 0:67 0:69 12:62 12:48 0:63 0:70 12:58 12:69 12:64 12:43 0:67 0:72 12:43 0:75 12:50 0:71 12:52 0:76 12:57 0:84 12:76 12:50 0:65 0:70 12:48 0:73 6:74 6:58 0:77 12:60 0:69 12:58 0:77 12:59 0:74 12:60 0:78 12:56 0:72 6:74 6:48 0:65 12:64 12:46 0:66 12:62 12:48 0:63 12:46 0:74 12:58 12:60 12:60 0:69 12:53 0:64 12:48 0:74 6:75 6:58 0:82 12:62 12:69 12:63 12:41 0:57 0:77 12:58 0:67 12:69 12:48 0:66 12:65 12:43 0:55 0:72 12:70 12:50 0:60 0:67 0:69 12:46 12:62 12:67 12:58 12:57 0:74 12:48 0:62 0:65 12:67 12:50 0:62 12:48 0:64 12:58 0:60 12:62 12:50 0:62 12:52 0:63 12:54 0:57 0:69 12:55 12:48 12:55 0:74 12:55 0:70 12:55 0:67 12:58 0:69 12:43 0:55 12:70 12:51 0:58 0:67 12:51 12:58 12:60 12:50 0:63 12:46 0:74 12:51 12:60 12:63 12:63 12:60 12:58 12:41 0:62 12:43 0:55 0:79 12:48 0:57 0:60 0:79 12:50 12:55 12:55 0:76 12:50 0:64 0:67 12:53 0:6

Translation:	0:65 0:69 6:60 6:65 6:64 0:67 0:69 12:60 6:64 6:58 6:57 6:62 0:67 12:60 6:53 6:60 6:69 6:65 6:69 6:72 6:76 6:65 6:67 6:70 6:65 6:69 6:64 6:58 6:69 6:62 0:70 6:67 6:69 6:60 6:58 6:76 6:69 6:60 6:58 6:65 6:69 6:57 6:65 6:65 6:58 0:64 0:70 6:65 0:74 6:64 0:74 12:62 6:57 6:65 6:63 6:64 0:70 0:76 6:74 6:69 0:76 6:70 0:75 6:69 0:75 6:67 0:79 6:60 6:62 6:64 0:67 6:64 6:62 6:69 0:74 6:65 6:63 6:62 0:74 6:62 6:69 6:65 6:74 6:76 6:65 6:68 6:62 6:65 6:59 6:67 6:70 6:65 6:62 0:74 6:72 6:58 6:70 6:67 6:65 6:64 6:58 6:65 6:72 6:67 6:63 6:53 6:65 6:69 6:72 6:74 6:69 6:75 6:64 6:77 6:67 6:77 6:76 6:64 6:76 6:73 6:73 6:75 6:73 6:60 6:70 6:76 6:64 6:60 6:57 0:76 6:65 6:69 6:65 6:60 0:74 6:76 6:55 6:77 6:76 6:70 6:68 6:79 6:57 0:77 6:81 6:55 0:79 6:81 6:81 6:64 0:70 6:72 0:76 6:55 6:57 6:72 0:77 6:67 6:55 6:79 6:64 6:76 6:65 6:72 6:57 0:75 6:73 6:60 6:74 6:53 0:69 6:75 6:60 6:81 6:64 6:70 0:72 6:65 0:72 6:59 6:72 0:77 6:72 6:62 6:77 6:52 6:60 6:82 6:65 0:69 6:72 0:76 6:62 6:77 6:65 0:69 6:55

Translation:	0:77 6:76 6:74 0:77 6:55 0:76 6:74 6:67 0:79 6:76 6:81 6:71 6:72 0:84 6:77 6:88 6:55 0:81 0:84 6:81 6:62 0:83 6:72 6:60 0:81 6:64 6:69 6:72 6:76 6:69 6:72 0:81 6:77 6:59 0:80 6:57 6:79 6:62 0:79 6:81 6:71 6:62 6:65 0:84 6:77 6:72 6:77 6:77 6:74 0:86 6:76 6:72 0:86 6:81 6:71 6:81 0:84 6:60 0:86 6:62 6:69 6:79 6:62 0:84 6:62 0:77 6:57 0:89 6:69 6:76 0:84 6:55 0:76 6:67 6:64 0:83 3:81 3:59 0:81 3:83 3:45 0:91 3:64 3:67 3:69 3:71 3:69 3:67 0:84 3:83 3:72 0:83 3:81 3:59 0:88 6:62 3:67 3:62 0:89 6:57 0:81 6:69 0:81 0:84 6:69 6:67 0:77 6:71 6:74 6:65 0:83 6:69 0:85 6:83 6:72 0:91 6:64 0:86 3:86 3:62 0:84 3:81 3:69 0:80 3:78 3:68 0:77 6:62 0:88 6:74 0:86 6:50 0:66 6:64 0:67 6:55 0:76 6:67 0:74 6:64 0:79 6:64 0:88 6:74 6:71 0:79 6:50 0:69 0:81 6:79 6:72 6:81 6:47 0:77 0:79 6:69 0:84 6:48 0:67 0:83 3:81 3:64 0:86 6:60 3:79 3:81 3:71 0:74 3:81 3:60 0:79 3:79 3:50 0:81 6:65 0:84 6:81 6:72 6:69 0:79 6:55 0:65 0:67 0:74 6:69 6:62 0:71 6:64 0:76 0:79 6:83 6:79 6:50 0:72 0:76 0:83 6:67 6:

Translation:	0:53 0:57 0:72 12:53 6:53 12:57 12:55 0:57 6:72 6:52 0:76 6:50 0:69 6:67 6:48 0:67 6:60 0:72 6:64 6:53 0:77 6:48 6:57 0:79 12:60 12:53 6:72 6:52 0:79 6:81 6:59 0:79 6:77 6:56 0:75 6:72 6:53 0:77 12:58 12:48 6:74 6:60 0:76 12:67 12:55 6:72 6:57 0:64 0:76 12:55 12:57 12:57 6:74 6:56 0:76 6:80 6:61 0:81 6:62 0:77 6:58 0:74 6:79 6:60 6:79 6:77 6:52 6:58 0:81 12:61 12:52 6:77 6:55 0:79 12:55 6:76 6:57 0:77 12:56 6:81 6:64 0:77 6:76 6:61 0:73 12:50 12:48 0:64 0:67 12:72 6:74 6:57 0:76 12:53 0:60 0:79 6:77 6:56 6:84 6:52 0:83 12:50 0:57 6:72 6:52 0:79 12:48 0:67 6:72 6:46 0:73 12:49 0:68 0:77 6:76 6:56 0:77 12:54 12:50 12:51 12:51 0:55 0:70 12:72 12:53 0:81 6:80 6:60 0:70 6:81 6:52 0:83 12:55 12:60 6:79 6:50 0:78 12:48 0:55 12:51 12:50 0:54 0:73 12:52 0:57 0:72 6:74 6:53 6:72 6:65 0:68 6:71 6:56 0:70 6:69 6:67 0:69 6:65 6:64 0:72 12:62 0:65 12:48 12:59 6:76 6:52 0:74 6:74 6:53 0:71 12:51 0:60 12:53 0:69 6:77 6:43 0:50 0:74 0:77 12:45 0:53 0:69 6:72 6:47 0:52 0:74 12:48 0:67 6:74 

...

Translation:	0:73 12:73 12:73 12:73 12:76 12:81 12:82 12:64 0:70 0:74 12:72 12:70 12:71 12:67 12:68 6:69 3:70 3:70 6:68 12:65 12:61 0:66 0:68 12:66 0:68 0:73 12:69 12:69 6:66 3:65 3:65 6:62 12:65 12:61 0:66 0:70 12:66 6:69 6:69 6:73 12:69 12:74 6:73 6:71 0:76 6:77 6:74 0:79 6:79 6:79 12:74 6:76 3:72 3:67 6:64 12:65 12:65 6:62 3:63 3:55 0:61 0:61 12:62 6:64 6:60 6:69 12:64 12:65 0:69 0:76 12:77 12:60 0:78 12:64 12:65 12:66 12:65 6:63 3:62 3:54 0:58 0:64 6:62 6:64 12:62 6:64 12:62 6:69 3:66 3:43 0:69 0:70 6:67 6:55 6:58 12:64 6:64 12:53 0:65 6:64 6:55 0:64 12:62 6:58 0:62 0:64 18:55 0:60 0:63 6:59 0:61 0:63 0:67 12:65 12:55 0:64 12:62 12:55 0:62 12:67 6:69 6:69 3:71 3:67 0:69 0:73 6:70 6:57 0:74 6:76 6:76 6:60 0:74 6:76 6:55 0:67 6:72 6:58 0:72 3:74 3:56 0:79 3:78 3:61 12:64 12:74 6:69 3:70 3:52 0:72 3:69 3:66 6:62 12:69 12:64 0:72 6:66 0:71 6:65 0:72 6:62 0:72 6:70 6:66 6:67 12:55 0:70 6:63 6:58 6:63 6:65 6:67 6:66 6:58 0:74 6:65 6:56 0:64 0:72 6:59 6:62 6:63 6:62 0:70 12:63 6:62 0:69 6

Translation:	0:57 0:60 0:65 48:57 0:60 48:53 0:58 0:63 30:53 0:57 0:65 24:55 0:58 0:67 18:55 0:58 0:65 18:57 0:61 6:55 0:58 6:53 0:55 6:50 0:54 0:57 24:50 6:53 0:60 6:62 6:43 0:50 0:55 24:58 0:65 6:65 6:48 0:57 12:53 0:60 6:53 12:48 12:53 0:57 12:55 0:65 12:55 12:55 6:65 0:67 12:48 0:52 24:53 0:57 6:53 6:53 0:58 12:48 0:64 12:53 0:60 12:48 12:41 0:48 24:48 24:53 0:57 12:65 12:60 12:48 12:60 12:64 12:58 0:67 6:67 12:53 0:58 12:53 0:55 12:48 12:57 12:53 0:57 12:46 0:57 0:62 24:46 0:53 0:58 12:65 12:53 0:58 12:41 12:53 12:41 0:55 24:53 0:55 24:53 0:57 24:41 6:48 12:53 6:55 0:58 24:55 0:60 12:52 0:60 12:55 12:58 12:50 12:57 12:60 12:48 0:57 0:65 0:69 0:72 24:62 12:60 0:72 12:64 12:46 0:60 0:62 24:48 24:58 0:65 12:48 12:60 0:65 24:48 0:55 6:60 12:60 0:64 12:55 12:43 0:62 0:65 24:57 24:60 12:62 12:59 12:57 0:60 12:46 0:62 24:58 12:62 0:67 12:53 12:53 12:48 0:53 0:57 12:60 12:60 12:48 6:60 12:60 6:58 12:57 0:65 12:50 12:60 12:48 0:57 24:60 12:48 24:53 12:53 12:48 6:55 6:48 0:57 24:46 0:55 12:

Translation:	0:53 0:60 36:53 0:60 48:53 48:53 0:65 6:53 0:69 0:72 36:53 0:67 0:70 12:60 0:65 36:60 6:48 0:55 30:60 6:55 0:64 0:67 0:72 30:60 6:62 12:55 12:55 12:60 48:53 6:65 0:69 6:65 6:65 6:55 0:59 6:71 6:71 6:71 6:60 6:53 6:67 12:67 6:65 12:53 0:60 12:53 6:65 6:69 12:50 0:60 6:65 6:50 12:60 6:53 12:55 12:60 18:55 12:48 0:60 24:55 6:64 12:55 0:65 6:67 18:55 6:60 12:53 0:55 12:65 0:72 6:69 12:65 12:53 12:50 0:55 0:71 12:74 6:65 12:48 12:55 0:65 6:67 6:48 0:60 36:55 0:62 24:55 6:65 6:55 0:71 12:60 6:72 6:69 12:60 6:48 6:67 6:67 12:64 12:53 0:65 12:67 12:67 12:65 12:43 0:71 6:67 6:67 12:55 0:67 24:60 6:62 6:60 6:55 6:65 0:72 12:48 6:60 6:69 6:60 6:65 6:64 12:43 0:55 6:64 0:72 12:60 12:41 0:48 0:53 24:48 0:67 6:67 6:65 6:48 6:55 6:60 12:53 12:48 6:55 6:64 0:67 12:55 6:64 6:55 6:67 6:55 0:72 12:53 12:65 0:69 6:67 6:48 0:65 6:64 12:64 12:47 0:53 12:62 12:47 0:50 12:60 12:47 0:47 12:62 6:65 12:47 0:50 12:67 12:47 0:69 12:71 12:41 0:45 12:69 12:41 0:50 12:67 12:65 12:52 0:62 12:53 0:57 0:64 

Translation:	0:57 0:71 3:72 2:76 4:71 4:69 4:56 2:69 5:57 0:69 0:72 6:52 0:71 3:72 3:57 3:69 4:56 2:69 4:56 1:71 5:72 4:69 3:57 2:72 4:57 0:72 5:52 4:57 3:52 0:71 6:50 0:74 6:76 6:74 6:55 0:71 6:60 6:64 6:60 3:64 1:72 3:76 3:64 3:74 5:76 3:55 1:71 3:60 2:72 3:55 3:52 6:50 6:55 6:50 6:50 0:67 6:53 0:68 6:52 6:56 0:64 0:71 6:55 0:68 6:54 0:57 6:55 0:68 0:71 0:74 6:59 6:67 6:57 3:71 1:68 2:64 3:59 6:69 6:57 0:64 6:69 6:72 6:60 6:53 0:71 3:72 3:56 0:69 3:52 6:68 6:56 6:56 0:71 6:55 6:68 6:64 3:72 3:64 6:74 6:60 0:64 0:72 0:76 6:67 6:64 6:57 0:71 6:60 0:72 6:52 0:64 6:59 0:67 6:55 0:68 6:55 0:59 0:67 0:71 6:53 6:55 0:60 3:67 3:55 0:59 6:55 6:59 3:55 3:55 6:57 6:60 0:71 6:55 0:71 6:56 6:68 6:62 6:59 6:60 0:64 0:68 6:71 3:64 3:55 0:72 3:67 3:50 6:55 6:50 0:68 6:51 6:53 0:65 6:50 6:55 6:43 0:60 0:64 12:48 0:57 0:72 12:48 0:57 0:69 0:74 12:48 0:54 0:65 0:74 6:55 6:60 6:62 0:67 12:55 6:62 6:57 6:59 0:65 3:67 3:53 0:71 6:55 0:71 6:58 6:53 0:55 0:59 0:67 6:57 6:65 3:67 3:58 6:56 0:68 3:67 3:60 0:6

Translation:	0:48 0:72 3:63 3:60 3:65 3:70 3:68 3:48 0:72 3:64 3:65 3:56 3:55 3:56 0:68 3:59 3:64 0:72 3:52 0:73 3:50 3:53 3:52 0:76 3:49 0:76 3:57 0:73 3:73 3:61 3:69 3:73 3:76 3:80 3:49 0:81 3:69 3:64 3:57 3:61 3:61 3:62 0:71 3:68 3:45 0:81 3:68 3:48 0:76 3:52 3:58 3:59 3:55 3:56 3:62 0:68 3:55 3:59 0:71 3:62 3:64 0:73 3:66 3:46 0:72 3:46 3:60 3:57 3:59 3:54 3:62 0:72 3:55 0:70 3:62 3:70 0:76 3:50 0:80 3:51 3:58 3:60 3:52 3:62 0:77 3:61 3:58 0:80 3:50 0:79 3:52 0:74 3:58 3:52 0:72 3:61 3:67 0:81 3:56 0:79 3:50 0:84 3:57 3:51 0:70 0:79 3:57 3:59 0:69 3:54 3:52 0:64 3:50 3:49 0:69 3:61 3:52 0:72 3:55 3:52 0:68 0:72 3:55 3:60 0:74 3:56 3:60 0:77 3:55 0:75 3:63 3:60 0:74 3:51 0:71 3:63 3:62 3:56 3:52 0:70 3:51 3:52 3:45 0:69 3:47 3:55 3:45 0:66 3:49 3:49 3:45 3:43 0:71 3:49 3:52 0:67 3:61 3:53 0:76 3:47 3:49 3:48 0:81 3:48 3:42 3:52 0:68 3:56 3:46 0:62 3:57 3:47 0:72 3:43 0:74 3:55 3:45 0:61 3:58 3:53 0:70 3:59 3:56 0:74 3:59 3:54 3:52 0:82 3:48 3:52 0:68 3:53 3:46 0:74 3:57 3:58 3:56 3: